In [ ]:
!pip install azure-identity azure-storage-file-datalake

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()  # Loads values from .env into os.environ

In [ ]:
from azure.identity import ClientSecretCredential, DefaultAzureCredential
from azure.storage.filedatalake import DataLakeServiceClient
import os

def get_adls_client(storage_account_name: str,
                    tenant_id: str = None,
                    client_id: str = None,
                    client_secret: str = None,
                    use_managed_identity: bool = False):
    """
    Returns a DataLakeServiceClient to interact with ADLS Gen2

    Parameters:
    - storage_account_name: name of the ADLS account
    - tenant_id, client_id, client_secret: for Service Principal (used in local/dev)
    - use_managed_identity: if True, uses DefaultAzureCredential (use in Azure)

    Returns:
    - DataLakeServiceClient
    """

    if use_managed_identity:
        credential = DefaultAzureCredential()
    else:
        credential = ClientSecretCredential(
            tenant_id=tenant_id,
            client_id=client_id,
            client_secret=client_secret
        )

    url = f"https://{storage_account_name}.dfs.core.windows.net"
    return DataLakeServiceClient(account_url=url, credential=credential)


In [ ]:

# for local environment , Values from your Azure App Registration (Service Principal)
client = get_adls_client(
    storage_account_name="your_adls_account_name",
    tenant_id=os.getenv("AZURE_TENANT_ID"),
    client_id=os.getenv("AZURE_CLIENT_ID"),
    client_secret=os.getenv("AZURE_CLIENT_SECRET"),
    use_managed_identity=False
)


In [ ]:

# Hosted on Azure , Azure will handle auth via Managed Identity
# client or datalake_client
client = get_adls_client(
    storage_account_name="your_adls_account_name",
    use_managed_identity=True
)


In [ ]:
# list files in the Directory

file_system_client = client.get_file_system_client(file_system="my-container")
paths = file_system_client.get_paths(path="my-folder")

for path in paths:
    print(f"Name: {path.name} | Directory: {path.is_directory}")

In [ ]:
# Connect to the container (like mounting a folder)
file_system_client = client.get_file_system_client("my-container")

# Access a file inside the folder
file_client = file_system_client.get_file_client("myfolder/sample.txt")

# Read the file
data = file_client.download_file().readall().decode("utf-8")
print(data)
